In [ ]:
!pip install torch torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# Define the autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 784),
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)

# Train the autoencoder
autoencoder = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

for epoch in range(10):  # Adjust the number of epochs as needed
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.view(inputs.size(0), -1)
        optimizer.zero_grad()
        _, outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()

# Extract features using the encoder
encoder = autoencoder.encoder

# Define the classification head
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(128, 10)  # Adjust the output size for your classification task

    def forward(self, x):
        x = self.fc(x)
        return x

# Load the MNIST dataset again for classification
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True, transform=transform),
    batch_size=64, shuffle=True
)

# Combine the encoder and classifier
classifier = Classifier()
optimizer_classifier = optim.Adam(classifier.parameters(), lr=0.001)
# Lists to store training history
train_loss_history = []
train_acc_history = []
 
for epoch in range(10):  # Adjust the number of epochs as needed
    running_loss = 0.0
    correct = 0
    total = 0
 
    for data in train_loader:
        inputs, labels = data
        inputs = inputs.view(inputs.size(0), -1)
        optimizer_classifier.zero_grad()
        encoded = encoder(inputs)
        outputs = classifier(encoded)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer_classifier.step()
 
        # Calculate training loss
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
 
    train_loss = running_loss / len(train_loader)
    train_accuracy = 100.0 * correct / total
    train_loss_history.append(train_loss)
    train_acc_history.append(train_accuracy)
 
    print(f'Epoch [{epoch + 1}/10], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%')
 
# Plot training loss and accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(range(1, 11), train_loss_history, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
 
plt.subplot(1, 2, 2)
plt.plot(range(1, 11), train_acc_history, label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()
 
# Now, you can use the combined model (encoder + classifier) for classification.

## Pre-trained:

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Load the MNIST dataset
(train_images, train_labels), (_, _) = keras.datasets.mnist.load_data()
train_images = train_images / 255.0  # Normalize pixel values to the range [0, 1]

# Resize the images to (224, 224) and add a channel dimension
train_images = tf.image.resize(train_images[..., tf.newaxis], (224, 224))




In [2]:
# Add a channel dimension
train_images = train_images[..., tf.newaxis]

# Convert grayscale to RGB
train_images_rgb = tf.image.grayscale_to_rgb(train_images)

ResourceExhaustedError: {{function_node __wrapped__Tile_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[60000,224,224,1,3] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu [Op:Tile] name: 

In [ ]:

# Create a TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=60000).batch(64)

# Load a pre-trained ResNet-18 model (excluding the final classification layer)
base_model = tf.keras.applications.ResNet50(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
)
base_model.trainable = False  # Freeze the pre-trained layers

# Add a custom classification head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10, activation='softmax')  # Adjust the output size for your classification task
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model (fine-tuning)
history = model.fit(train_dataset, epochs=10)

# Plot training loss and accuracy
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()
